In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

food_data = pd.read_csv('/content/food-dataset-extended-2.csv')

rating_columns = ["spice level", "sweet level", "sour level", "salt level", "bitter level"]
food_data[rating_columns] = food_data[rating_columns] / 10.0

ratings = food_data[rating_columns].mean(axis=1).values

dishes = np.arange(len(food_data))
X_train, X_test, y_train, y_test = train_test_split(dishes, ratings, test_size=0.2, random_state=42)

dish_input = Input(shape=(1,), name="dish_input")

dish_embedding = Embedding(input_dim=len(food_data), output_dim=16, name="dish_embedding")(dish_input)
dish_embedding = Flatten()(dish_embedding)

x = Dense(64, activation='relu')(dish_embedding)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, activation='relu')(x)

output = Dense(1, activation='sigmoid', name="output_layer")(x)

ncf_model = Model(inputs=dish_input, outputs=output)
ncf_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

ncf_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

ncf_model.summary()

def get_user_preferences():
    cuisine = 'Indian'
    taste = 'Sweet'
    temperature = "Hot"
    spice_level = 1 / 10.0
    sweet_level = 8 / 10.0
    salt_level = 1 / 10.0
    sour_level = 3 / 10.0
    bitter_level = 1 / 10.0

    return {
        "cuisine": cuisine.lower(),
        "taste": taste.lower(),
        "temperature": temperature.lower(),
        "ratings": np.array([spice_level, sweet_level, salt_level, sour_level, bitter_level]).mean()
    }

def recommend_dishes(n=5):
    user_prefs = get_user_preferences()

    filtered_data = food_data[(food_data['Cuisine'].str.lower() == user_prefs['cuisine']) &
                              (food_data['taste'].str.lower() == user_prefs['taste']) &
                              (food_data['temperature'].str.lower() == user_prefs['temperature'])]

    if filtered_data.empty:
        print("No exact matches found. Showing top-rated dishes instead.")
        filtered_data = food_data

    dish_indices = np.arange(len(filtered_data))

    predicted_scores = ncf_model.predict(dish_indices)

    top_n_indices = np.argsort(predicted_scores.flatten())[::-1][:n]

    print("Top", n, "recommended dishes:")
    print(filtered_data.iloc[top_n_indices])

recommend_dishes(n=5)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0260 - mae: 0.1357 - val_loss: 0.0221 - val_mae: 0.1214
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0226 - mae: 0.1223 - val_loss: 0.0214 - val_mae: 0.1178
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0253 - mae: 0.1288 - val_loss: 0.0215 - val_mae: 0.1174
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0205 - mae: 0.1122 - val_loss: 0.0216 - val_mae: 0.1176
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0201 - mae: 0.1081 - val_loss: 0.0217 - val_mae: 0.1185
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0140 - mae: 0.0840 - val_loss: 0.0232 - val_mae: 0.1211
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0072 - mae: 0.0595 - val_loss: 0.0251 - val_mae: 0.1285
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0059 - mae: 0.0568 - val_loss: 0.0265 - val_mae: 0.1318
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0050 - mae

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dish_input (InputLayer)              │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dish_embedding (Embedding)           │ (None, 1, 16)               │           5,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,981 (105.40 KB)

 Trainable params: 8,993 (35.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,988 (70.27 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Top 5 recommended dishes:
    Cuisine     Dish name          Dish  taste temperature  spice level  \
283  Indian       biryani       Biryani  Sweet         Hot          0.2   
120  Indian       payasam       Payasam  Sweet         Hot          0.2   
20   Indian  shahi paneer  Shahi Paneer  Sweet         Hot          0.2   
58   Indian        jalebi        Jalebi  Sweet         Hot          0.1   
121  Indian        kesari        Kesari  Sweet         Hot          0.1   

     sweet level  salt level  sour level  bitter level  
283          0.3         NaN         0.2           0.1  
120          0.7         0.2         NaN           NaN  
20           0.4         NaN         NaN           NaN  
58           0.9         NaN         NaN           NaN  
121          0.7         0.2         NaN           NaN  


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split
import numpy as np

rating_columns = ["spice level", "sweet level", "sour level", "salt level", "bitter level"]
food_data[rating_columns] = food_data[rating_columns] / 10.0

ratings = food_data[rating_columns].mean(axis=1).values

dishes = np.arange(len(food_data))

X_train, X_test, y_train, y_test = train_test_split(dishes, ratings, test_size=0.2, random_state=42)

dish_input = Input(shape=(1,), name="dish_input")

dish_embedding = Embedding(input_dim=len(food_data), output_dim=16, name="dish_embedding")(dish_input)
dish_embedding = Flatten()(dish_embedding)

x = Dense(64, activation='relu')(dish_embedding)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, activation='relu')(x)

output = Dense(1, activation='sigmoid', name="output_layer")(x)

ncf_model = Model(inputs=dish_input, outputs=output)
ncf_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

ncf_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

def get_user_preferences():
    '''cuisine = 'Chinese'
    taste = 'Sweet'
    temperature = "Cold"
    spice_level = 1 / 10.0
    sweet_level = 8 / 10.0
    salt_level = 1 / 10.0
    sour_level = 3 / 10.0
    bitter_level = 1 / 10.0'''

    cuisine = input("Cuisine preference (e.g., Indian, Chinese, American): ")
    taste = input("Taste preference (e.g., Sweet, Sour, Bitter): ")
    temperature = input("Temperature (e.g., Hot, Cold): ")
    spice_level = float(input("Spice level (1-10): ")) / 10.0
    sweet_level = float(input("Sweet level (1-10): ")) / 10.0
    salt_level = float(input("Salt level (1-10): ")) / 10.0
    sour_level = float(input("Sour level (1-10): ")) / 10.0
    bitter_level = float(input("Bitter level (1-10): ")) / 10.0

    return {
        "cuisine": cuisine.lower(),
        "taste": taste.lower(),
        "temperature": temperature.lower(),
        "ratings": np.array([spice_level, sweet_level, salt_level, sour_level, bitter_level]).mean()
    }

def recommend_dishes(n=5):
    user_prefs = get_user_preferences()

    filtered_data = food_data[(food_data['Cuisine'].str.lower() == user_prefs['cuisine']) &
                              (food_data['taste'].str.lower() == user_prefs['taste']) &
                              (food_data['temperature'].str.lower() == user_prefs['temperature'])]

    if filtered_data.empty:
        print("No exact matches found. Showing top-rated dishes instead.")
        filtered_data = food_data

    dish_indices = np.arange(len(filtered_data))

    predicted_scores = ncf_model.predict(dish_indices)

    top_n_indices = np.argsort(predicted_scores.flatten())[::-1][:n]

    print("\nTop", n, "Recommended Dishes:\n")
    for dish in filtered_data.iloc[top_n_indices]['Dish name']:
        print(f"- {dish}")

recommend_dishes(n=5)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.2375 - mae: 0.4871 - val_loss: 0.1931 - val_mae: 0.4394
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1729 - mae: 0.4147 - val_loss: 0.1033 - val_mae: 0.3212
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0768 - mae: 0.2726 - val_loss: 0.0239 - val_mae: 0.1539
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0147 - mae: 0.1158 - val_loss: 0.0040 - val_mae: 0.0628
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0023 - mae: 0.0437 - val_loss: 0.0014 - val_mae: 0.0372
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0010 - mae: 0.0282 - val_loss: 8.5503e-04 - val_mae: 0.0286
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.7331e-04 - mae: 0.0223 - val_loss: 6.0816e-04 - val_mae: 0.0240
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.4175e-04 - mae: 0.0200 - val_loss: 4.5054e-04 - val_mae: 0.0206
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step -